Coding challenge
This page consists a coding challenge for Data Engineering roles at Changing Room.

Purpose
Aim of this test is three fold

- evaluate your coding abilities
- judge your technical experience
- understand how you design a solution

How you will be judged:

You will be scored on,
-coding standard, comments and style
-unit testing strategy
-overall solution design
-appropriate use of source control

Instructions

Candidate should put their test results on a public code repository hosted on Github
Once test is completed please share the Github repository URL to hiring team so they can review your work.
Create a solution that crawls for fashion products information, stores in a PostGRESQL database.
Bonus points:
- if you store the data on an AWS RDS
- if you are able to deploy the scraper using AWS dockers

Details
Write an application to crawl an online fashion brand website, e.g. https://www.fordays.com, https://www.reformation.com or https://www.zara.com using a crawler framework such as Selenium, bs4, etc. You can use a crawl framework of your choice in Python. (YOU ONLY NEED TO SCRAPE A FEW PRODUCTS, not entire website, however, please explain your strategy to scrape the whole website, extract all the URLs and update the database automatically overtime (new products, update old products not available anymore)

this is an example of how the extracted information should be structured:

- display_name (str)
- product_material (str)
- color (str)
- size (list)
- price (str)
- product_url (str)
- image_links (list)
- brand_name (str)
- description (str)
- scrapped_date (date)
- low_level (str) [category of clothes: e.g. casual pants, dress]
- gender (str) [men, women, or kids]
- secondhand (bool) [is it from a second hand retailer, already worn?]


Store the data in a hosted PostGRES database.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import date
from sqlalchemy import create_engine
import psycopg2
import time

In [2]:
# I try to save the data in my local database 
DB_HOST = '127.0.0.1'
DB_NAME = 'testdb'
DB_USER = 'postgres'
DB_PASS = 'root'
DB_PORT = '5432'
DB = "testdb"

In [3]:
#I plan to use BeautifulSoup to get the data and store them into a dataframe in pandas 
def get_items(url = 'https://fordays.com/collections/all-women', verbose = True):
    start_time = time.time()
    col = ['product_id','display_name', 'product_material','color','size','price',
       'product_url','image_links','brand_name','description','scrapped_date',
      'low_level','gender','secondhand']
    data = pd.DataFrame(columns = col)
    
    response = requests.get(url)
    if not response.status_code == 200:
        print('status code: ',response.status_code)
        return None
    try:
        results_page = BeautifulSoup(response.content,'lxml')
        
        div_tags = results_page.find_all('div',{"class":"grid-product"})
        
        for i in range(len(div_tags)):
            display_name = div_tags[i].attrs['data-product-title']

            try:
                color = ','.join(div_tags[i].find_all('div')[-1].get_text().replace('\n',' ').split())
            except:
                color = ''

            price = div_tags[i].find_all('div')[0].find('div',class_='price-container').get_text().strip()
            product_url = 'https://fordays.com/' + div_tags[i].find_all('div')[0].find('a').get('href')
            image_links = 'https:' + div_tags[i].find('img').get('data-src').format(width= '720x')
            
            product_id = int(div_tags[i].attrs['data-product-id'])

            res = requests.get(product_url)

            product_page = BeautifulSoup(res.content,'lxml')
            brand_name = 'fordays'
            size = product_page.find('fieldset',{'name':'Size'}).get_text().replace('Size','').strip().replace('\n',',')

            description = product_page.find('div', class_= 'product-single__description rte').get_text().strip()
            scrapped_date = date.today()
            product_material = None
            low_level = None
            gender = 'women'
            secondhand = False

            item = [product_id,display_name, product_material,color,size,price,
                   product_url,image_links,brand_name,description,scrapped_date,
                  low_level,gender,secondhand]
            data.loc[len(data)] =item
        
        sql_engine = create_engine("postgresql://" + DB_USER + ":" + DB_PASS + "@" + DB_HOST +'/'+ DB)
        data.to_sql('fordays', con = sql_engine, if_exists='replace',index = False)
        end_time = time.time()
        elapsed_time = (end_time-start_time)
        if verbose:
            print('Get success\nelapsed time is {} sec'.format(elapsed_time))
        return data
    except:
        return None

In [4]:
#test
def test():
    sql_engine = create_engine("postgresql://" + DB_USER + ":" + DB_PASS + "@" + DB_HOST +'/'+ DB)
    conn = psycopg2.connect(database= DB, user=DB_USER, password=DB_PASS, host=DB_HOST , port=DB_PORT) 
    cur = conn.cursor()
    res = cur.execute("select * from fordays")
    return cur.fetchall()

In [5]:
def updatedb(url = 'https://fordays.com/collections/all-women'):
    sql_engine = create_engine("postgresql://" + DB_USER + ":" + DB_PASS + "@" + DB_HOST +'/'+ DB)
    start_time = time.time()
    new_data = get_items(verbose = False)
    
    #I try to delete all the product with the same product id as those in the new dataframe
    try:
        cur.execute('delete from fordays where product_id in ({})'.format(','.join([str(i) for i in new_data.product_id.tolist()])))
        conn.commit()
    except:
        pass
    
    try:
        new_data.to_sql('fordays', con = sql_engine, if_exists='append',index = False)
        print("update success")
        end_time = time.time()
        elapsed_time = (end_time-start_time)
        print("elapsed time is {} sec".format(elapsed_time))
    except:
        print("update failure")
        

In [6]:
data = get_items()

Get success
elapsed time is 9.876688957214355 sec


In [7]:
test()

[(6286090469528,
  'Organic Cotton Daily Tee',
  None,
  'FRENCHIE,PALOMA,ECRU,CELADON,DARK,GREEN,BLUELIGHT,CINNAMON,GREY,WHITE,CAPETOWN,BLACK',
  'XS,S,M,L,XL,XXL',
  '$28',
  'https://fordays.com//collections/all-women/products/daily-crew',
  'https://cdn.shopify.com/s/files/1/0503/2601/2056/products/S01218W_FRENCHIE_0028_720xx.jpg?v=1649446760',
  'fordays',
  'Live out your days in our Organic Cotton Daily Tee. It’s completely lightweight, organic cotton that’s soft to the touch and made to last. Airy and updated, it’s a flattering take on the ever-adored tee.',
  datetime.date(2022, 4, 11),
  None,
  'women',
  False),
 (7220472971416,
  'Recycled Terry Fleece Hoodie',
  None,
  'PALOMA,LINEN,BLACK',
  'XS,S,M,L,XL,XXL',
  '$88',
  'https://fordays.com//collections/all-women/products/loop-back-hoodie',
  'https://cdn.shopify.com/s/files/1/0503/2601/2056/products/320028U_PALOMA_720xx.jpg?v=1649273101',
  'fordays',
  'The Recycled Terry Fleece Hoodie is an upgrade to your favorite 

In [8]:
updatedb()

update success
elapsed time is 8.234641075134277 sec
